Функция, лемматизирующая текст (и изначально очищала от стоп-слов, но так не работал один из методов позднее)

In [1]:
from string import punctuation
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import os

PUNCT = punctuation + '-'
morph = MorphAnalyzer()
SW = stopwords.words('russian')

In [2]:
def lemmatize(my_line):
    my_line = my_line.translate(str.maketrans('', '', PUNCT))
    words = my_line.split()
    lem_words = [morph.parse(w.replace('‘', '').replace('’', ''))[0].normal_form for w in words]
    #filtered = [w for w in lem_words if w not in SW]
    return lem_words #filtered

В качестве корпуса используем сайт школы лингвистики, раздел с новостями (рубрика "наука", т.к. там обычно больше тегов стоит). Автоматически собираем ссылки, заголовки, текст новости и теги. Для каждой новости создаём словарь, где хранятся все данные, связанные с ней (туда же будем сохранять автоматические ключевые слова разных методов)

In [3]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import re
import time
import csv
from itertools import zip_longest

user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'

In [4]:
tokens = 0
texts = 0
data = []

req = urllib.request.Request('https://ling.hse.ru/news/science/', headers = {'User-Agent':user_agent})
with urllib.request.urlopen(req) as response:
    html = response.read()
soup = BeautifulSoup(html, 'html.parser')
news = soup.find_all('div', {'class': "post__content"})
tag_set = soup.find_all('div', {'class': "tag-set"})
for i in range(len(news)):
    
    if tokens > 5000:
        break
    
    news_link = news[i].find('a').get('href')
    news_title = news[i].find('a').get('title')
    tags = [tag.get('title') for tag in tag_set[i+1].find_all('a', {'class': "tag rubric rubric--tag"})]
    
    if 'ling' in news_link:# новости с других источников не парсятся так же
        req = urllib.request.Request(news_link, headers = {'User-Agent':user_agent})
        with urllib.request.urlopen(req) as response:
            html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        text = soup.find('div', {'class': "post__text"}).text
        lemmas_text = lemmatize(news_title + text)
        
        news_dict = dict()
        news_dict['link'] = news_link
        news_dict['title'] = news_title
        news_dict['auto_tags'] = tags
        news_dict['text'] = text
        news_dict['lemmas'] = lemmas_text
        
        data.append(news_dict)
        tokens += len(lemmas_text)
        texts += 1

Проверяем размер корпуса

In [5]:
print(texts, 'текстов,', tokens, 'токенов')

8 текстов, 4285 токенов


Добавляем размеченные вручную ключевые слова. Проблема тегов на сайте шл в том, что не все из этих слов реально есть в тексте, поэтому эталоном станет объединение ключевых слов, выделенных вручную, и тех тегов, в которых все слова тега есть в тексте.

In [6]:
my_tags = ['западнодвинская экспедиция, диалектологическая экспедиция, тверская область, местные жители, деревня, шетнево, макеево, разговор, лексика, студенты',
 'honorable mention, Нияз Киреев, студент, премия, конференция, Мемориальная премия имени Чарльза Таунсэнда',
 'доклад, иранские языки, конференция, Институт лингвистики РГГУ',
 'Иван Саркисов, диссертация, защита, тайская и бирманская поэзия',
 'Ольга Драгой, Анна Журавлева, центр языка и мозга, грант, аспирант, картирование речи, конкурс, Научный центр "Идея"',
 'фольклорная экспедиция, костромская область, Большие Рымы, Макарьевский район, унжлаг, традиции, жители, валенки, обряды, стихи, верования, исследование',
 'НУГ, иранские языки, конференция, доклад',
 'Нияз Киреев, студент, конференция, SLS Annual Meetings, доклад, славистика, пленарный доклад, Фридман, фонетика, Зализняк'
]
iter_tags = iter(my_tags)

for new in data:
    new['my_tags'] = next(iter_tags).split(', ')
    my_tags_lemmas = [tuple(lemmatize(w)) for w in new['my_tags']]
    auto_tags_lemmas = [tuple(lemmatize(w)) for w in new['auto_tags']]
    text_words = []
    for tag in auto_tags_lemmas:
        s = 0
        for word in tag:
            if word in new['lemmas']:
                s += 1
        if s == len(tag):
            text_words.append(tag)
    new['key_words'] = list(set(my_tags_lemmas).union(set(text_words)))

Методами автоматического выделения будут RAKE, TextRank и Tf-Idf. Устанавливаем всё необходимое и делаем матрицу tf-idf.

In [7]:
#!pip3 install python-rake
#!pip3 install summa
import RAKE
rake = RAKE.Rake(SW)
from summa import keywords

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

corpus = []
for new in data:
    corpus.append(' '.join(new['lemmas']))

matrix = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names_out()

Добавляем результаты автоматического выделения в словари и одновременно с этим создаём фильтрованные варианты. Я заметила, что в ключевых словах часто оказываются, например, глаголы, причём не самые содержательные, поэтому в качестве фильтра оставим только ключевые слова, состоящие из существительного

In [9]:
matr_iter = iter(matrix)

for new in data:
    text = ' '.join(new['lemmas'])
    
    rake_words = rake.run(text, maxWords=2, minFrequency=1)[:15]
    new['rake'] = [tuple(key_word[0].split()) for key_word in rake_words]
    new['rake_filtered'] = [key_word for key_word in new['rake'] if len(key_word) == 1 and morph.parse(key_word[0])[0].tag.POS == 'NOUN']
    
    textrank_words = keywords.keywords(text, language='russian', additional_stopwords=SW, scores=True)[:15]
    new['textrank'] = [tuple(key_word[0].split()) for key_word in textrank_words]
    new['textrank_filtered'] = [key_word for key_word in new['textrank'] if len(key_word) == 1 and morph.parse(key_word[0])[0].tag.POS == 'NOUN']
    
    tfidf_words = [x for _, x in sorted(zip(next(matr_iter).toarray()[0], terms), reverse=True) if x not in SW][:15]
    new['tf-idf'] = [tuple(key_word.split()) for key_word in tfidf_words]
    new['tf-idf_filtered'] = [key_word for key_word in new['tf-idf'] if len(key_word) == 1 and morph.parse(key_word[0])[0].tag.POS == 'NOUN']

Считаем precision и recall

In [10]:
for new in data:
    for method in ['rake', 'textrank', 'tf-idf']:
        intersection = len(set(new['key_words']).intersection(set(new[method])))
        filt_intersection = len(set(new['key_words']).intersection(set(new[method + '_filtered'])))
    
        if intersection == 0:
            new[method + '_precision'] = 0
            new[method + '_filt_precision'] = 0
            new[method + '_recall'] = 0
            new[method + '_filt_recall'] = 0
        elif filt_intersection == 0:
            new[method + '_precision'] = intersection/len(new[method])
            new[method + '_filt_precision'] = 0
            new[method + '_recall'] = intersection/len(new['key_words'])
            new[method + '_filt_recall'] = 0
        else:
            new[method + '_precision'] = intersection/len(new[method])
            new[method + '_filt_precision'] = filt_intersection/len(new[method + '_filtered'])
            new[method + '_recall'] = intersection/len(new['key_words'])
            new[method + '_filt_recall'] = filt_intersection/len(new['key_words'])

Считаем средний показатель этих метрик по текстам и f1-score, смотрим, что получилось

In [11]:
print('Precision, filt. Precision; Recall, filt. Recall; F1, filt. F1')
for method in ['rake', 'textrank', 'tf-idf']:
    precision = round(sum([new[method + '_precision'] for new in data])/8, 2)
    filt_precision = round(sum([new[method + '_filt_precision'] for new in data])/8, 2)
    recall = round(sum([new[method + '_recall'] for new in data])/8, 2)
    filt_recall = round(sum([new[method + '_filt_recall'] for new in data])/8, 2)
    f1score = round(2*precision*recall/(precision + recall), 2)
    filt_f1score = round(2*filt_precision*filt_recall/(filt_precision + filt_recall), 2)
    print(f'{method}: {precision}, {filt_precision}; {recall}, {filt_recall}; {f1score}, {filt_f1score}')

Precision, filt. Precision; Recall, filt. Recall; F1, filt. F1
rake: 0.09, 0.13; 0.15, 0.06; 0.11, 0.08
textrank: 0.08, 0.19; 0.16, 0.16; 0.11, 0.17
tf-idf: 0.15, 0.23; 0.31, 0.28; 0.2, 0.25


In [12]:
print('Эталон: ', data[0]['key_words'], '\n')
print('RAKE: ', data[0]['rake'], '\n')
print('TextRank: ', data[0]['textrank'], '\n')
print('TF-IDF: ', data[0]['tf-idf'], '\n')

Эталон:  [('разговор',), ('диалектологический', 'экспедиция'), ('лексика',), ('тверская', 'область'), ('деревня',), ('местный', 'житель'), ('студент',), ('бакалавриат',), ('шетнево',), ('макеево',), ('западнодвинский', 'экспедиция')] 

RAKE:  [('тверская', 'область'), ('смоленский', 'говор'), ('образ', 'исследователь'), ('точно', 'уверенный'), ('работа', 'например'), ('действительность', 'встречаться'), ('это', 'утратить'), ('вооружиться', 'опросник'), ('план', 'работа'), ('свой', 'дом'), ('проблема', 'решить'), ('находиться', 'недалеко'), ('довольно', 'необычный'), ('необычный', 'часы'), ('маша', 'замбржицкий')] 

TextRank:  [('экспедиция',), ('год',), ('деревня',), ('большой',), ('больший',), ('говор', 'местный', 'житель'), ('ещё',), ('сбор',), ('всё',), ('час',), ('часы',), ('говорить',), ('который',), ('нужно',), ('птица',)] 

TF-IDF:  [('деревня',), ('экспедиция',), ('житель',), ('год',), ('местный',), ('макеево',), ('шетнево',), ('птица',), ('прошлый',), ('ещё',), ('лексика',), (

Что можно заметить? У rake было довольно много тегов из нескольких слов (в том числе удачных, совпадающих с эталоном), поэтому при фильтрации recall сильно падает, а precision немного вырастает. При этом остальные два метода выделяют не выделяют или почти не выделяют таких ключевых фраз, поэтому при фильтрации уходит много глаголов и других частей речи, и precision заметно вырастает. В целом f1-score для фильтрованного варианта скорее выше (и даже для RAKE не сильно падает, несмотря на заметно пониженный recall). Лучше всего работает TF-IDF!

Из лишнего выделяются некоторые случайные словосочетания и слова ("большой", "ещё", "в действительности встречаются"). Возможно, автор просто использует одну и ту же конструкцию для описания, поэтому она распознается как повторяющаяся и значимая. Это можно попробовать решить, например, более полным фильтром по структуре тега (чтобы ключевые фразы с глаголами или состоящие из одного прилагательного не попадали в итог); также можно было бы дополнить список стоп-слов, это помогло бы избавиться от совсем незначимых слов ("всё", "который")


Не выделяются названия: название конференции может быть упомянуто всего один раз (или в разных формулировках, с аббревиатурой и без и т.п.), но оно значимо для текста. Это мог бы решить дополнительный этап, где в тексте бы выделялись именованные сущности и прибавлялись к ключевым словам (возможно, не все, а как-то отфильтрованные)

Для русского есть некоторая проблема с однокоренными словами (в тексте есть "верования", "вера", "верить", но при этом каждое из слов встречается не так часто и как следствие не выделяется как ключевое). Может можно попробовать выделять корень (делать стемминг?) или считать расстояние Левенштейна и считать общую встречаемость для всех однокоренных слов.